In [2]:
# Clear Environment 

rm(list = ls())

# Predicting Insurance Fraud (Rohil Rao)

In [11]:
## Loading Required libraries;

library(caret)
library(DMwR)
library(h2o)
library(ROCR)
library(ggplot2)
library(vegan)

Warning message:
"package 'vegan' was built under R version 3.6.3"Loading required package: permute
Warning message:
"package 'permute' was built under R version 3.6.3"This is vegan 2.5-7

Attaching package: 'vegan'

The following object is masked from 'package:caret':

    tolerance



#### Reading dataset

In [14]:
## Read The Train and Test Data into Enviornment and defining NAs found in the dataset
 
train_data1 = read.csv(file = "Train.csv", header = T, sep = ',')
train_data2 = read.csv(file = "Train_Claim.csv", header = T, sep = ',',na.strings = c('?','-5','MISSINGVALUE','MISSEDDATA'))
train_data3 = read.csv(file = "Train_Demographics.csv", header = T, sep = ',')
train_data4 = read.csv(file = "Train_Policy.csv", header = T, sep = ',',na.strings = c('-1','MISSINGVAL','NA'))
train_data5 = read.csv(file = "Train_Vehicle.csv", header = T, sep = ',',na.strings = '???')
  
  
test_data1 = read.csv(file = "Test.csv", header = T, sep = ',')
test_data2 = read.csv(file = "Test_Claim.csv", header = T, sep = ',',na.strings = c('?','-5','MISSINGVALUE','MISSEDDATA'))
test_data3 = read.csv(file = "Test_Demographics.csv", header = T, sep = ',')
test_data4 = read.csv(file = "Test_Policy.csv", header = T, sep = ',',na.strings = c('-1','MISSINGVAL','NA'))
test_data5 = read.csv(file = "Test_Vehicle.csv", header = T, sep = ',',na.strings = '???')


#### Summarize the data and check summary statistics

In [21]:
# Summarize the data
summary(train_data1)
#summary(train_data2)
#summary(train_data3)
#summary(train_data4)
#summary(train_data5)
str(train_data1)
#str(test_data2)
#str(test_data3)
#str(test_data4)
#str(test_data5)

     CustomerID    ReportedFraud
 Cust10000:    1   N:21051      
 Cust10001:    1   Y: 7785      
 Cust10002:    1                
 Cust10003:    1                
 Cust10004:    1                
 Cust10005:    1                
 (Other)  :28830                

'data.frame':	28836 obs. of  2 variables:
 $ CustomerID   : Factor w/ 28836 levels "Cust10000","Cust10001",..: 8317 22720 11897 25444 27130 1332 15235 25726 15908 13215 ...
 $ ReportedFraud: Factor w/ 2 levels "N","Y": 1 1 1 2 2 1 2 1 1 1 ...


In [22]:

###### train vehicle data pre processing #######
train_VehicleID = subset(train_data5, VehicleAttribute == "VehicleID")
train_VehicleMake = subset(train_data5, VehicleAttribute == "VehicleMake")
train_VehicleModel = subset(train_data5, VehicleAttribute == "VehicleModel")
train_VehicleYOM = subset(train_data5, VehicleAttribute == "VehicleYOM")
  
  
  
df1 = merge(x = train_VehicleID, y = train_VehicleMake, by = "CustomerID", all = TRUE)
df2 = merge(x = train_VehicleModel, y = train_VehicleYOM, by = "CustomerID", all = TRUE)
train_data5_final <- merge(x = df1, y = df2 , by = "CustomerID", all =TRUE)
train_data5_final <- train_data5_final[,!(colnames(train_data5_final) %in% c("VehicleAttribute.x.x","VehicleAttribute.y.x","VehicleAttribute.x.y","VehicleAttribute.y.y"))]
names(train_data5_final) <- c("CustomerID","VehicleID","VehicleMake","VehicleModel","VehicleYOM")
nrow(train_data5_final)
str(train_data5_final)

[1] 28836

'data.frame':	28836 obs. of  5 variables:
 $ CustomerID  : Factor w/ 28836 levels "Cust10000","Cust10001",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ VehicleID   : Factor w/ 28910 levels "1995","1996",..: 13763 4643 25320 22338 15151 13339 1674 13817 1954 15055 ...
 $ VehicleMake : Factor w/ 28910 levels "1995","1996",..: 30 30 28907 28907 69 69 56 66 28907 64 ...
 $ VehicleModel: Factor w/ 28910 levels "1995","1996",..: 27 27 50 50 37 37 32 32 60 23 ...
 $ VehicleYOM  : Factor w/ 28910 levels "1995","1996",..: 14 12 5 9 16 17 6 16 1 10 ...


In [23]:

##Converting datatypes to adjust for the differenet levels in Vehicle data
  
train_data5_final$VehicleID = as.factor(as.character(train_data5_final$VehicleID))
train_data5_final$VehicleMake = as.factor(as.character(train_data5_final$VehicleMake))
train_data5_final$VehicleModel = as.factor(as.character(train_data5_final$VehicleModel))
train_data5_final$VehicleYOM = as.factor(as.character(train_data5_final$VehicleYOM))
str(train_data5_final)

'data.frame':	28836 obs. of  5 variables:
 $ CustomerID  : Factor w/ 28836 levels "Cust10000","Cust10001",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ VehicleID   : Factor w/ 28836 levels "Vehicle10000",..: 13693 4573 25250 22268 15081 13269 1604 13747 1884 14985 ...
 $ VehicleMake : Factor w/ 14 levels "Accura","Audi",..: 2 2 14 14 13 13 9 12 14 11 ...
 $ VehicleModel: Factor w/ 39 levels "3 Series","92x",..: 6 6 21 21 12 12 8 8 29 2 ...
 $ VehicleYOM  : Factor w/ 21 levels "1995","1996",..: 14 12 5 9 16 17 6 16 1 10 ...


In [24]:
  
##Similarly doing pre processing on test vehicle data
  
###### test vehicle data pre processing #######
  
  
  
test_VehicleID = subset(test_data5, VehicleAttribute == "VehicleID")
test_VehicleMake = subset(test_data5, VehicleAttribute == "VehicleMake")
test_VehicleModel = subset(test_data5, VehicleAttribute == "VehicleModel")
test_VehicleYOM = subset(test_data5, VehicleAttribute == "VehicleYOM")
  
df3 = merge(x = test_VehicleID, y = test_VehicleMake, by = "CustomerID", all = TRUE)
df4 = merge(x = test_VehicleModel, y = test_VehicleYOM, by = "CustomerID", all = TRUE)
  
test_data5_final <- merge(x = df3, y = df4 , by = "CustomerID", all =TRUE)
test_data5_final <- test_data5_final[,!(colnames(test_data5_final) %in% c("VehicleAttribute.x.x","VehicleAttribute.y.x","VehicleAttribute.x.y","VehicleAttribute.y.y"))]
names(test_data5_final) <- c("CustomerID","VehicleID","VehicleMake","VehicleModel","VehicleYOM")
nrow(test_data5_final)
str(test_data5_final)
  

  
##Adjusting for levels
  
test_data5_final$VehicleID = as.factor(as.character(test_data5_final$VehicleID))
test_data5_final$VehicleMake = as.factor(as.character(test_data5_final$VehicleMake))
test_data5_final$VehicleModel = as.factor(as.character(test_data5_final$VehicleModel))
test_data5_final$VehicleYOM = as.factor(as.character(test_data5_final$VehicleYOM))
str(test_data5_final)
  

[1] 9662

'data.frame':	9662 obs. of  5 variables:
 $ CustomerID  : Factor w/ 9662 levels "Cust10008","Cust10010",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ VehicleID   : Factor w/ 9735 levels "1995","1996",..: 6694 1912 320 8999 7443 1830 6292 5309 5414 7189 ...
 $ VehicleMake : Factor w/ 9735 levels "1995","1996",..: 9732 59 65 56 29 38 65 38 68 68 ...
 $ VehicleModel: Factor w/ 9735 levels "1995","1996",..: 60 69 48 39 67 62 43 58 36 36 ...
 $ VehicleYOM  : Factor w/ 9735 levels "1995","1996",..: 1 12 16 13 15 13 16 18 12 13 ...
'data.frame':	9662 obs. of  5 variables:
 $ CustomerID  : Factor w/ 9662 levels "Cust10008","Cust10010",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ VehicleID   : Factor w/ 9662 levels "Vehicle10010",..: 6625 1843 251 8930 7374 1761 6223 5240 5345 7120 ...
 $ VehicleMake : Factor w/ 14 levels "Accura","Audi",..: 14 10 12 9 1 5 12 5 13 13 ...
 $ VehicleModel: Factor w/ 38 levels "3 Series","92x",..: 29 35 20 13 34 31 16 28 11 11 ...
 $ VehicleYOM  : Factor w/ 21 levels "1995","1996",..: 1 12 

# Merging the dataset

In [25]:
  
## Merging the data
  
train_data=Reduce(function(x, y) merge(x, y, all=TRUE), list(train_data1,train_data2,train_data3,train_data4,train_data5_final))
df=Reduce(function(x,y) merge(x,y, all=TRUE), list(test_data2,test_data3,test_data4,test_data5_final))
test_data1$id=1:nrow(test_data1)
test_datax=merge(test_data1,df,by = "CustomerID",sort = FALSE)
test_data=test_datax[order(test_datax$id),]

sum(is.na(train_data))

train_data <- centralImputation(data = train_data)
test_data <- centralImputation(data = test_data)

[1] 25774

In [28]:
dim(train_data)

[1] 28836    42

In [29]:
write.csv(train_data, file = "train_data.csv")
write.csv(test_data, file = "test_data.csv")